# Sprint1課題　機械学習フロー

## 【問題1】クロスバリデーション
事前学習期間は検証用データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。

具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。

[sklearn.model_selection.KFold — scikit-learn 0.20.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)



## 【問題2】グリッドサーチ
これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。

グリッドサーチをパイプラインの中に組み込みましょう。

**※問題1と問題2の内容を同じプログラム内に組み込んでいます**

In [32]:
 """
 ライブラリのインポート
 """
import numpy as np

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

import seaborn as sns
import pandas as pd

pd.set_option('display.max_rows', 500)

In [50]:
# train.csvの読み込み
df_train = pd.read_csv("application_train.csv")

In [54]:
# EXT_SOURCRに着目する
X_train_EX = df_train.loc[:, ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]

# それぞれの欠損値に対しては平均代入法を利用する
X_train_EX["EXT_SOURCE_1"] = X_train_EX["EXT_SOURCE_1"].fillna(X_train_EX["EXT_SOURCE_1"].mean())
X_train_EX["EXT_SOURCE_2"] = X_train_EX["EXT_SOURCE_2"].fillna(X_train_EX["EXT_SOURCE_2"].mean())
X_train_EX["EXT_SOURCE_3"] = X_train_EX["EXT_SOURCE_3"].fillna(X_train_EX["EXT_SOURCE_3"].mean())

#numpyに変更
X= X_train_EX.values
y = df_train['TARGET']

In [55]:
#StratifiedKFoldの設定
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

# KFoldの設定
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

In [56]:
#パラメータよりクロスバリデーション、グリッドサーチを実行する
def cv_gs_skf(estimators, parameters):

    # plを一つの分類器としてみなす
    pl = Pipeline(estimators)

    # 分類器を渡し、グリッドサーチのインスタンス生成
    clf = GridSearchCV(pl, parameters, n_jobs=-1, cv=3)
    
    print("<StratifiedKFold>")
    for train_index, test_index in skf.split(X, y):

        clf.fit(X[train_index], y[train_index]) 

        # ベストパラメータを出力
        print('Best_estimator = {0}'.format(clf.best_params_))

        # auc算出
        lr_auc = roc_auc_score(y[test_index], clf.predict_proba(X[test_index])[:, 1])
        print("classifier_auc: {}".format(lr_auc))

In [57]:
#パラメータよりクロスバリデーション、グリッドサーチを実行する
def cv_gs_kf(estimators, parameters):

    # plを一つの分類器としてみなす
    pl = Pipeline(estimators)

    # 分類器を渡し、グリッドサーチのインスタンス生成
    clf = GridSearchCV(pl, parameters, n_jobs=-1, cv=3)
    
    print("<KFold>")
    for train_index, test_index in kf.split(X, y):

        # 優れたハイパーパラメーターを探索
        clf.fit(X[train_index], y[train_index])

        # ベストパラメータを出力
        print('Best_estimator = {0}'.format(clf.best_params_))

        # auc算出
        lr_auc = roc_auc_score(y[test_index], clf.predict_proba(X[test_index])[:, 1])
        print("classifier_auc: {}".format(lr_auc))


### ロジスティック回帰でクロスバリデーション&グリッドサーチ

In [132]:
# PCAで次元削減、ロジスティック回帰での分類
estimators = [('pca', PCA()),
              ('lr', LogisticRegression())]

# グリッドサーチでの探索パラメータ
parameters = {"pca__n_components" : range(2, 3),
              "lr__penalty" : ["l2", "l1"],
              'lr__C': np.logspace(-5, 2, 10).tolist(), 
              'lr__solver' : ['liblinear', 'saga']
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_kf(estimators, parameters)

<StratifiedKFold>
Best_estimator = {'lr__C': 0.0774263682681127, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'pca__n_components': 2}
classifier_auc: 0.7160107649299055
Best_estimator = {'lr__C': 1e-05, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'pca__n_components': 2}
classifier_auc: 0.7120298466816025
Best_estimator = {'lr__C': 0.4641588833612782, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'pca__n_components': 2}
classifier_auc: 0.7197282371087573
<KFold>
Best_estimator = {'lr__C': 0.0774263682681127, 'lr__penalty': 'l1', 'lr__solver': 'liblinear', 'pca__n_components': 2}
classifier_auc: 0.7161065322942142
Best_estimator = {'lr__C': 1e-05, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'pca__n_components': 2}
classifier_auc: 0.711820492892695
Best_estimator = {'lr__C': 0.01291549665014884, 'lr__penalty': 'l1', 'lr__solver': 'saga', 'pca__n_components': 2}
classifier_auc: 0.7197081757226581


### K近傍法でクロスバリデーション&グリッドサーチ

In [126]:
#KNeighborsClassifierのインポート
from sklearn.neighbors import KNeighborsClassifier


# PCAで次元削減、ロジスティック回帰での分類
estimators = [('pca', PCA()),
              ('kn', KNeighborsClassifier())]

# グリッドサーチでの探索パラメータ
parameters = {"pca__n_components" : range(2, 3),
              "kn__n_neighbors" : [30, 40, 50, 60, 70, 80]
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_kf(estimators, parameters)

<StratifiedKFold>
Best_estimator = {'kn__n_neighbors': 30, 'pca__n_components': 2}
classifier_auc: 0.6676570894220896
Best_estimator = {'kn__n_neighbors': 50, 'pca__n_components': 2}
classifier_auc: 0.6834363543028925
Best_estimator = {'kn__n_neighbors': 70, 'pca__n_components': 2}
classifier_auc: 0.6956131954543117
<KFold>
Best_estimator = {'kn__n_neighbors': 30, 'pca__n_components': 2}
classifier_auc: 0.6674848615518618
Best_estimator = {'kn__n_neighbors': 50, 'pca__n_components': 2}
classifier_auc: 0.6831681990226973
Best_estimator = {'kn__n_neighbors': 70, 'pca__n_components': 2}
classifier_auc: 0.696003084983554


### LinearSVCでクロスバリデーション&グリッドサーチ
カーネル法有りのSVMでは処理が終わらない。。

In [131]:
#LinearSVM
from sklearn.svm import SVC
from sklearn.svm import LinearSVC


# PCAで次元削減、ロジスティック回帰での分類
estimators = [('pca', PCA()),
              ('svc', LinearSVC())]

# グリッドサーチでの探索パラメータ
parameters = {"pca__n_components" : range(2, 3),
              'svc__C' : np.logspace(-7, 3, 9).tolist(), 
              "svc__loss" : ['hinge', 'squared_hinge'],               
              'svc__random_state' : [1]
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_kf(estimators, parameters)

<StratifiedKFold>
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7148936945698176
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7120135823895499
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7182900452165455
<KFold>
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7150504299084475
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7118030288664075
Best_estimator = {'pca__n_components': 2, 'svc__C': 1e-07, 'svc__loss': 'hinge', 'svc__random_state': 1}
classifier_auc: 0.7183891636752728


### 決定木でクロスバリデーション&グリッドサーチ

In [150]:
# DecisionTreeClassifierのインポート
from sklearn.tree import DecisionTreeClassifier

estimators = [('pca', PCA()),
              ('dtc', DecisionTreeClassifier())]

# グリッドサーチでの探索パラメータ
parameters = {"pca__n_components" : range(2, 3),
              'dtc__criterion' : ['gini', 'entropy'], 
              "dtc__min_samples_split" : range(3000, 5000, 100)
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_kf(estimators, parameters)

<StratifiedKFold>
Best_estimator = {'dtc__criterion': 'entropy', 'dtc__min_samples_split': 4800, 'pca__n_components': 2}
classifier_auc: 0.707682443865701
Best_estimator = {'dtc__criterion': 'gini', 'dtc__min_samples_split': 4800, 'pca__n_components': 2}
classifier_auc: 0.7064086081478116
Best_estimator = {'dtc__criterion': 'gini', 'dtc__min_samples_split': 4900, 'pca__n_components': 2}
classifier_auc: 0.7133792554589261
<KFold>
Best_estimator = {'dtc__criterion': 'entropy', 'dtc__min_samples_split': 4900, 'pca__n_components': 2}
classifier_auc: 0.7079778810564739
Best_estimator = {'dtc__criterion': 'gini', 'dtc__min_samples_split': 4900, 'pca__n_components': 2}
classifier_auc: 0.7092478394685067
Best_estimator = {'dtc__criterion': 'entropy', 'dtc__min_samples_split': 4900, 'pca__n_components': 2}
classifier_auc: 0.7140699018865326


### ランダムフォレストでクロスバリデーション&グリッドサーチ

In [155]:
#RandomForestClassifierのインポート
from sklearn.ensemble import RandomForestClassifier

estimators = [('pca', PCA()),
              ('rfc', RandomForestClassifier())]

# グリッドサーチでの探索パラメータ
parameters = {"pca__n_components" : range(2, 3),
              'rfc__n_estimators' : [100], 
              'rfc__min_samples_split' : [4000]
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_kf(estimators, parameters)


<StratifiedKFold>
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.7159504432843572
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.7140886313502693
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.720657926579575
<KFold>
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.7160470858304945
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.7135468823189142
Best_estimator = {'pca__n_components': 2, 'rfc__min_samples_split': 4000, 'rfc__n_estimators': 100}
classifier_auc: 0.7206130278066046


#### わずかであるが、ランダムフォレストのスコア(AUC)が高い結果を得られた。

## 【問題3】Kernelからの調査
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。

### 下記の特徴量で検証
1. ３つのEXT_SOURCEに対して重み付けをつけた特徴量
1. ３つのEXT_SOURCEにおける行ごとの最小値
1. ３つのEXT_SOURCEにおける行ごとの最大値
1. "DAYS_EMPLOYED"(重要度が高い)  
1. "AMT_CREDIT"(重要度が高い)


In [84]:
X_train_EX2 = pd.DataFrame([])

#重み付け
a = 2
b = 9
c = 4

#重み付けした
X_train_EX2['EXT_SOURCE_WEIGH'] = (
    X_train_EX['EXT_SOURCE_1'] * a + 
    X_train_EX['EXT_SOURCE_2'] * b + 
    X_train_EX['EXT_SOURCE_3'] * c
) / (a + b + c)

X_train_EX2['EXT_SOURCE_MAX'] = X_train_EX.max(axis='columns')
X_train_EX2['EXT_SOURCE_MIN'] = X_train_EX.min(axis='columns')
X_train_EX2['DAYS_EMPLOYED'] = df_train["DAYS_EMPLOYED"]
X_train_EX2['AMT_CREDIT'] = df_train["AMT_CREDIT"]

#numpyに変更
X = X_train_EX2.values

In [89]:
#RandomForestClassifierのインポート
from sklearn.ensemble import RandomForestClassifier

#StandardScalerをインポート
from sklearn.preprocessing import StandardScaler


clf = RandomForestClassifier(min_samples_split=4000, n_estimators=100)

print("<StratifiedKFold>")
for train_index, test_index in skf.split(X, y):
    
    #インスタンス生成
    scaler = StandardScaler()

    #学習用データに対してfit
    #渡されたデータの最大値、最小値、平均、標準偏差、傾き...などの統計を取得して、内部メモリに保存する。
    scaler.fit(X[train_index])

    #学習用・検証用データにに対してtransform
    #fit()で取得した統計情報を使って、渡されたデータを実際に書き換える。
    X_train_transform = scaler.transform(X[train_index])
    X_test_transform = scaler.transform(X[test_index])

    clf.fit(X[train_index], y[train_index]) 

    # auc算出
    lr_auc = roc_auc_score(y[test_index], clf.predict_proba(X[test_index])[:, 1])
    print("classifier_auc: {}".format(lr_auc))

<StratifiedKFold>
classifier_auc: 0.7244520052408318
classifier_auc: 0.7264711391043173
classifier_auc: 0.7218832497842361
classifier_auc: 0.7283765565495782
classifier_auc: 0.7289927880290354


In [90]:
# test.csvの読み込み
df_test = pd.read_csv("application_test.csv")
df_test.shape

(48744, 121)

In [96]:
'''
testデータの処理
'''
# EXT_SOURCEに着目する
X_test_EX = df_test.loc[:, ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]

# それぞれの欠損値に対しては平均代入法を利用する
X_test_EX["EXT_SOURCE_1"] = X_test_EX["EXT_SOURCE_1"].fillna(X_test_EX["EXT_SOURCE_1"].mean())
X_test_EX["EXT_SOURCE_2"] = X_test_EX["EXT_SOURCE_2"].fillna(X_test_EX["EXT_SOURCE_2"].mean())
X_test_EX["EXT_SOURCE_3"] = X_test_EX["EXT_SOURCE_3"].fillna(X_test_EX["EXT_SOURCE_3"].mean())

X_test_EX2 = pd.DataFrame([])

#重み付け
a = 2
b = 10
c = 7

#重み付けした
X_test_EX2['EXT_SOURCE_WEIGH'] = (
    X_test_EX['EXT_SOURCE_1'] * a + 
    X_test_EX['EXT_SOURCE_2'] * b + 
    X_test_EX['EXT_SOURCE_3'] * c
) / (a + b + c)

X_test_EX2['EXT_SOURCE_MAX'] = X_test_EX.max(axis='columns')
X_test_EX2['EXT_SOURCE_MIN'] = X_test_EX.min(axis='columns')
X_test_EX2['DAYS_EMPLOYED'] = df_test["DAYS_EMPLOYED"]
X_test_EX2['AMT_CREDIT'] = df_test["AMT_CREDIT"]

#numpyに変更
X_test = X_test_EX2.values

In [97]:
# kaggle提出用のdataframeを作成する
submission = pd.DataFrame({'SK_ID_CURR': df_test['SK_ID_CURR'], 'TARGET': clf.predict_proba(X_test)[:, 1]})

In [98]:
# kaggle提出用のcsvファイルを作成する
submission.to_csv('HomeCreditDefaultRisk_2019043005.csv', index = False)

### kaggleへの提出結果
- Private Score : 0.70689
- Public Score : 0.71441

→train_data内でのクロスバリデーションよりも0.02〜0.03ほど低い結果となった。

## 【問題4】高い汎化性能のモデル
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。

### LightGBMを用いた検証
問題3で作成したランダムフォレストより汎化性能が高くなるか検証を行う。

In [99]:
import lightgbm as lgb
lgb.LGBMClassifier(num_leaves=31)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [100]:
estimators = [
              ('lgb', lgb.LGBMClassifier())]

# グリッドサーチでの探索パラメータ
parameters = {
              'lgb__num_leaves' : range(2,50,5)
             }

#パラメータよりクロスバリデーション、グリッドサーチを実行する
cv_gs_skf(estimators, parameters)

<StratifiedKFold>
Best_estimator = {'lgb__num_leaves': 7}
classifier_auc: 0.7281658047308912
Best_estimator = {'lgb__num_leaves': 22}
classifier_auc: 0.7299289251550388
Best_estimator = {'lgb__num_leaves': 27}
classifier_auc: 0.725271760558339
Best_estimator = {'lgb__num_leaves': 22}
classifier_auc: 0.7316000798771085
Best_estimator = {'lgb__num_leaves': 12}
classifier_auc: 0.7333050564379224


In [104]:
clf = lgb.LGBMClassifier(num_leaves=10)

print("<StratifiedKFold>")
for train_index, test_index in skf.split(X, y):
    
    clf.fit(X[train_index], y[train_index]) 

    # auc算出
    lr_auc = roc_auc_score(y[test_index], clf.predict_proba(X[test_index])[:, 1])
    print("classifier_auc: {}".format(lr_auc))

<StratifiedKFold>
classifier_auc: 0.728428405609937
classifier_auc: 0.7293840832624273
classifier_auc: 0.7270026075839685
classifier_auc: 0.7310837036894143
classifier_auc: 0.732328615607197


In [105]:
# kaggle提出用のdataframeを作成する
submission = pd.DataFrame({'SK_ID_CURR': df_test['SK_ID_CURR'], 'TARGET': clf.predict_proba(X_test)[:, 1]})

In [106]:
# kaggle提出用のcsvファイルを作成する
submission.to_csv('HomeCreditDefaultRisk_2019043007.csv', index = False)

### kaggleへの提出結果
- Private Score :　0.70838
- Public Score : 0.71503

→ランダムフォレストを使用した際よりもわずかに数値が上がったが、大きな変化はなし。  
　ランダムフォレストと同様、train_data内でのクロスバリデーションよりも0.02〜0.03ほど低い結果となった。

### まとめ
今回は利用するモデル・ハイパーパラメータの比較を重点的に実施。パラーメータの設定によりスコアに差異が発生することを確認し、そん中でクロスバリデーションを実施することができた。
モデルにより、処理時間に大きく違いがあり、今後の参考としたい。
今回特徴量の改善が部分的であったため、より大きな改善を行うためにも特徴量に対する検証も必要。